In [112]:
from parser import parse_input
import os
import pandas as pd
from pre_process_constraints import pre_process_constraints
from aux import *
from Cliques import *
from graph_builder import *
from plot_graphs import *

import gurobipy as gp
from gurobipy import GRB

In [113]:
current_instance="Inputs/instance_0004.txt"

In [114]:
best_A = solution_map.get(current_instance)
input_folder = "Inputs"
instance="instance_0004.txt"
instance_path = os.path.join(input_folder, instance)

In [115]:
# Assuming you've already parsed the instance
parsed_data = parse_input(current_instance)

# Convert to DataFrames for better visual inspection
orders_df = pd.DataFrame(parsed_data['orders'], columns=[f'Item {j}' for j in range(parsed_data['num_items'])])
aisles_df = pd.DataFrame(parsed_data['aisles'], columns=[f'Item {j}' for j in range(parsed_data['num_items'])])

# Add identifiers for readability
orders_df.index = [f'Order {i}' for i in range(parsed_data['num_orders'])]
aisles_df.index = [f'Aisle {i}' for i in range(parsed_data['num_aisles'])]

# Show the results
print("\n=== Orders Matrix ===")
print(orders_df)

print("\n=== Aisles Matrix ===")
print(aisles_df)

num_orders=parsed_data['num_orders']
num_aisles=parsed_data['num_aisles']
num_items=parsed_data['num_items']



=== Orders Matrix ===
          Item 0  Item 1  Item 2  Item 3  Item 4  Item 5  Item 6  Item 7  \
Order 0        0       0       0       0       0       0       1       0   
Order 1        0       0       0       0       0       0       0       0   
Order 2        0       0       0       0       0       0       0       0   
Order 3        0       1       0       0       0       0       0       0   
Order 4        0       0       0       0       0       0       0       0   
Order 5        0       0       0       0       0       0       0       0   
Order 6        0       0       0       0       0       0       0       0   
Order 7        0       0       1       0       0       0       0       0   
Order 8        0       0       0       0       0       0       0       0   
Order 9        0       0       0       0       0       1       0       1   
Order 10       0       0       0       0       0       0       0       0   
Order 11       0       0       0       0       1       0       0 

In [116]:
model_lp,X,Y=build_lp_model(parsed_data,best_A,clique_constraints=None,lp_filename="model.lp")
model_lp.write("lp_filename.lp")

     Building LP model...
     LP model successfully built with 0 clique constraints.


In [117]:
A,b=pre_process_constraints(parsed_data)

In [118]:
len(A)

61

In [119]:
len(b)

61

In [120]:
S=clique_detection(A,b)
G=graph_builder(S)
S_extended=clique_extension(G,S)
S_strenghted=remove_dominated_cliques(S_extended)

In [121]:
len(S_strenghted)

25

In [122]:
S_strenghted

[[0, 142],
 [0, 199],
 [2, 127],
 [3, 151],
 [4, 135],
 [8, 155],
 [9, 125],
 [9, 130],
 [9, 146],
 [9, 173],
 [9, 213],
 [11, 139],
 [11, 213],
 [12, 150],
 [14, 135],
 [133, 14],
 [169, 11],
 [177, 4],
 [177, 14],
 [190, 14],
 [192, 13],
 [193, 2],
 [193, 15],
 [200, 13],
 [203, 7]]

In [123]:
model_best,added_cliques,Xcut,Ycut=clique_cut_separator(parsed_data,
                                                            best_A,
                                                            G_all,
                                                            S_strenghted,
                                                            max_rounds=20,
                                                            min_viol=1e-4,
                                                            instance_name=instance)


🔁 ROUND 1
     Building LP model...
     LP model successfully built with 0 clique constraints.
⚠️ Violated clique: [0, 199], weight sum = 1.0049 > 1.0001
⚠️ Violated clique: [4, 135], weight sum = 1.0020 > 1.0001
⚠️ Violated clique: [11, 139], weight sum = 1.7391 > 1.0001
⚠️ Violated clique: [11, 213], weight sum = 1.4224 > 1.0001
⚠️ Violated clique: [14, 135], weight sum = 1.0005 > 1.0001
⚠️ Violated clique: [133, 14], weight sum = 1.0005 > 1.0001
⚠️ Violated clique: [169, 11], weight sum = 1.8003 > 1.0001
⚠️ Violated clique: [177, 4], weight sum = 1.0011 > 1.0001
⚠️ Violated clique: [190, 14], weight sum = 1.0005 > 1.0001
➕ 9 new clique cuts added.

🔁 ROUND 2
     Building LP model...
     LP model successfully built with 9 clique constraints.
⚠️ Violated clique: [203, 7], weight sum = 1.3078 > 1.0001
➕ 1 new clique cuts added.

🔁 ROUND 3
     Building LP model...
     LP model successfully built with 10 clique constraints.
🛑 No new violated cliques found.


In [124]:
len(added_cliques)

10

In [125]:
model_best.write("ip_filename.lp")

In [126]:
G_all=graph_builder(added_cliques)
plot_expression_graph(G=G_all,
                      num_orders=num_orders,
                      num_aisles=num_aisles,
                      cliques=added_cliques,
                      title="Cliques for instance 0004",
                      graph_layout="circular",
                      figsize=8,
                      nodesize=200,
                      filename='graph0004')

saved plot to: graph0004
